# Outgroup allele counts phase3 dataset creation

In this notebook I created the new dataset with allele counts for outgroups.
This notebook is structured in 5 steps:

1) <b>Loading step</b>, I loaded the Ag1000G phase2 pass data, the ingroup data and the outgroup data

2) <b>Alignment step</b>, I aligned the each outgroup genome with the phase3 pass dataset

3) <b>Writing step</b>, I wrote the new allele count of the outgroups on a zarr dataset

4) <b>Mapping step</b>, I mapped the outgroup alle count dataset to the phase3 biallelic daset to take only the biallelic variants for each outgroup

5) <b>Building step</b>, I wrote the new biallelic allele count of the outgroups in a new dataset


Import modules:

In [1]:
import os
import sys
import allel
import zarr
import pandas as pd
import petl as etl
from matplotlib import pyplot
import seaborn as sns
import h5py
import pyfasta
import numpy as np
nnz = np.count_nonzero

def log(*msg):
    print(' '.join(map(str, msg)), file=sys.stdout)
    sys.stdout.flush()
    
autosomes = '2R', '2L', '3R', '3L'
chromosomes = autosomes + ('X',)

/home/randomx/anaconda3/envs/pyenv1/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Import Alistair modules:

In [4]:
%run 'imports_20150407.ipynb'

-----------------------------

## 1) Loading step

Loading phase2 calldata:

In [10]:
import malariagen_data
ag3 = malariagen_data.Ag3("gs://vo_agam_release/")

---------------------------

Loading outgroup calldata:

In [6]:
outgroup_species = ['chri', 'epir']

In [7]:
outgroup_variants_fn_template = '/home/randomx/{species}_fake_cnvrt_sort.vcf.gz.vcfnp_cache/variants.{chrom}.npy'

--------------------------

## 2) Align step

Align outgroup calldata to the phase3 calldata:

In [37]:
def align_outgroup_ac(chrom, species):

    # load Ag1000G variant positions and alternate alleles
    #variants = callset[chrom]['variants']
    filter_pass = ag3.site_filters(chrom, mask="gamb_colu").compute()
    f = filter_pass[:]
    pos, ref, alt = ag3.snp_sites(chrom)
    pos = allel.SortedIndex(pos[f].compute())
    ref = ref[f].compute()
    alt= alt[f].compute()
    
    # load outgroup variant positions and alternate alleles
    variants_other = np.load(outgroup_variants_fn_template.format(species=species, chrom=chrom), mmap_mode='r')
    pos_other = allel.SortedIndex(variants_other['POS'], copy=False)
    alt_other = variants_other['ALT']

    # locate intersection between callsets
    loc_isec, loc_other_isec = pos.locate_intersection(pos_other)
    # exclude duplicates
    loc_other_dup = pos_other == np.roll(pos_other, 1)
    loc_other_isec &= ~loc_other_dup
    assert nnz(loc_isec) == nnz(loc_other_isec)
    log(pos.size, 'variants in Ag1000G')
    log(nnz(loc_isec), 'variants in intersection')

    # filter data to the intersection
    alt_isec = alt[loc_isec]
    alt_other_isec = alt_other[loc_other_isec]

    # setup array to store outgroup allele counts with alleles remapped to Ag1000G
    n_variants_isec = nnz(loc_isec)
    ac_am = np.zeros((n_variants_isec, 4), dtype='i4')
    
    # reference allele observed
    loc_ref = alt_other_isec == b'.'
    loc_a1 = alt_isec[:, 0] == alt_other_isec
    loc_a2 = alt_isec[:, 1] == alt_other_isec
    loc_a3 = alt_isec[:, 2] == alt_other_isec
    ac_am[loc_ref, 0] = 1
    ac_am[loc_a1, 1] = 1
    ac_am[loc_a2, 2] = 1
    ac_am[loc_a3, 3] = 1

    # finally extend to all Ag1000G variant positions
    ac_aligned = np.zeros((pos.shape[0], 4), dtype='i4')
    ac_aligned[loc_isec] = ac_am
    for i in range(4):
        log(i, nnz(ac_aligned[:, i]))
    
    return ac_aligned

christyi try:

In [38]:
ac_aligned = align_outgroup_ac('3L', 'chri')
ac_aligned

28707856 variants in Ag1000G
19274720 variants in intersection
0 15195564
1 1388820
2 1544082
3 1146254


array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       ...,
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=int32)

--------------------------------

## 3) Writing step

Write the align on a new hdf5 dataset:

In [39]:
with h5py.File('data/outgroup_allele_counts_phase3.h5',
               mode='a') as outgroup_allele_counts:
    for chrom in chromosomes:
        h5g = outgroup_allele_counts.require_group(chrom)
        #for species in ingroup_species:
         #   if species in h5g:
          #      log(chrom, species, 'skipping')
            #else:
             #   log(chrom, species, 'building')
               # ac_aligned = align_ingroup_ac(chrom, species)
              #  h5d = h5g.create_dataset(species, data=ac_aligned, chunks=True)

        for species in outgroup_species:
            if species in h5g:
                log(chrom, species, 'skipping')
            else:
                log(chrom, species, 'building')
                ac_aligned = align_outgroup_ac(chrom, species)
                h5d = h5g.create_dataset(species, data=ac_aligned, chunks=True)

2R chri building
44439759 variants in Ag1000G
32946127 variants in intersection
0 26016146
1 2375760
2 2630901
3 1923320
2R epir building
44439759 variants in Ag1000G
33142683 variants in intersection
0 25052652
1 2566405
2 3074483
3 2449143
2L chri building
36005131 variants in Ag1000G
25399023 variants in intersection
0 20028934
1 1833334
2 2032322
3 1504433
2L epir building
36005131 variants in Ag1000G
26024720 variants in intersection
0 19677290
1 2030362
2 2407878
3 1909190
3R chri building
37199402 variants in Ag1000G
26518390 variants in intersection
0 21033451
1 1876097
2 2080609
3 1528233
3R epir building
37199402 variants in Ag1000G
26806397 variants in intersection
0 20345542
1 2050246
2 2450998
3 1959611
3L chri building
28707856 variants in Ag1000G
19274720 variants in intersection
0 15195564
1 1388820
2 1544082
3 1146254
3L epir building
28707856 variants in Ag1000G
19666721 variants in intersection
0 14923702
1 1512199
2 1799095
3 1431725
X chri building
16362809 variant

----------------------------------------------

In [43]:
calldata_out= h5py.File('data/outgroup_allele_counts_phase3.h5', mode = 'r')

In [44]:
chri_2R = calldata_out['2R']['chri'][:]
chri_2L = calldata_out['2L']['chri'][:]
chri_3R = calldata_out['3R']['chri'][:]
chri_3L = calldata_out['3L']['chri'][:]
chri_X = calldata_out['X']['chri'][:]

In [45]:
epir_2R = calldata_out['2R']['epir'][:]
epir_2L = calldata_out['2L']['epir'][:]
epir_3R = calldata_out['3R']['epir'][:]
epir_3L = calldata_out['3L']['epir'][:]
epir_X = calldata_out['X']['epir'][:]

## 5) Writing step

In [48]:
chri_2R

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       ...,
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=int32)

In [49]:
root = zarr.open('data/outgroup_alleles_phase3.zarr', mode='a')

In [50]:
#foo = root.create_group('foo')
#bar = foo.create_dataset('bar', data=my_array)
# shortcuts
bar = root.create_dataset('2R/chri', data=chri_2R)
bar = root.create_dataset('2R/epir', data=epir_2R)

In [51]:
bar = root.create_dataset('2L/chri', data=chri_2L)
bar = root.create_dataset('2L/epir', data=epir_2L)

In [52]:
bar = root.create_dataset('3R/chri', data=chri_3R)
bar = root.create_dataset('3R/epir', data=epir_3R)

In [53]:
bar = root.create_dataset('3L/chri', data=chri_3L)
bar = root.create_dataset('3L/epir', data=epir_3L)

In [54]:
bar = root.create_dataset('X/chri', data=chri_X)
bar = root.create_dataset('X/epir', data=epir_X)

In [56]:
root.tree()

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, name='2L', nodes=(Node(disabled=True, ico…

----------------------------------
# Sandbox
## Ingroup dataset

Load ingroup calldata:

In [22]:
ingroup_species = 'arab', 'meru', 'mela', 'quad'
ingroup_callset_fn_template = '/bucket/outgroup/UnifiedGenotyper/{species}_ref_ug_vqsr_cnvrt_sort.h5'
agc_callsets = {species: h5py.File(ingroup_callset_fn_template.format(species=species), mode='r')
                    for species in ingroup_species}

Test mela:

In [23]:
agc_callsets['mela']

<HDF5 file "mela_ref_ug_vqsr_cnvrt_sort.h5" (mode r)>

Align ingroup calldata to the phase2 calldata:

In [26]:
def align_ingroup_ac(chrom, species):
    
    # load Ag1000G variant positions and alternate alleles
    #variants = callset[chrom]['variants']
    pos = allel.SortedIndex(ag3.snp_sites(chrom, field = "POS", site_mask="gamb_colu").compute())
    alt = ag3.snp_sites(chrom, field = "ALT", site_mask="gamb_colu").compute()
    
    # load ingroup variant positions and alternate alleles
    variants_other = agc_callsets[species][chrom]['variants']
    pos_other = allel.SortedIndex(variants_other['POS'][:], copy=False)
    alt_other = agc_callsets[species][chrom]['variants']['ALT'][:]
    
    # locate intersection between callsets
    loc_isec, loc_other_isec = pos.locate_intersection(pos_other)
    # exclude duplicates
    loc_other_dup = pos_other == np.roll(pos_other, 1)
    loc_other_isec &= ~loc_other_dup
    assert nnz(loc_isec) == nnz(loc_other_isec)
    log(pos.size, 'variants in Ag1000G')
    log(nnz(loc_isec), 'variants in intersection')
    
    # filter data to the intersection
    alt_isec = alt[loc_isec]
    alt_other_isec = alt_other[loc_other_isec]
    
    # load ingroup genotypes and count alleles
    genotype_other_isec = allel.GenotypeChunkedArray(agc_callsets[species][chrom]['calldata']['genotype']).compress(loc_other_isec, axis=0)
    ac_other_isec = genotype_other_isec.count_alleles()[:]

    # setup array to store ingroup allele counts with alleles remapped to Ag1000G
    n_variants_isec = nnz(loc_isec)
    ac_am = np.zeros((n_variants_isec, 4), dtype='i4')

    # fill in reference allele counts
    ac_am[:, 0] = ac_other_isec[:, 0]
    
    # fill in alternate allele counts
    loc_a1 = alt_isec[:, 0] == alt_other_isec
    loc_a2 = alt_isec[:, 1] == alt_other_isec
    loc_a3 = alt_isec[:, 2] == alt_other_isec
    ac_am[loc_a1, 1] = ac_other_isec[loc_a1, 1]
    ac_am[loc_a2, 2] = ac_other_isec[loc_a2, 1]
    ac_am[loc_a3, 3] = ac_other_isec[loc_a3, 1]    
    
    # finally extend to all Ag1000G variant positions
    ac_aligned = np.zeros((pos.shape[0], 4), dtype='i4')
    ac_aligned[loc_isec] = ac_am
    for i in range(4):
        log(i, nnz(ac_aligned[:, i]))
    
    return ac_aligned

In [27]:
ac_aligned = align_ingroup_ac('3L', 'mela')
ac_aligned

10640388 variants in Ag1000G
9769468 variants in intersection
0 9259374
1 330829
2 87100
3 8495


array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       ...,
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=int32)